In [1]:
import os
import logging
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [14]:
def GPT_API_text(prompt: str, user_input: str):
  # 初始化 Azure OpenAI 客戶端
    try:
        client = AzureOpenAI(
            api_key=os.getenv("4OAPIKey"),
            api_version="2024-08-01-preview",
            azure_endpoint=os.getenv("4OEndpoint")
        )
        # logging.info("Azure OpenAI 客戶端初始化成功")
    except Exception as e:
        logging.error(f"初始化 Azure OpenAI 客戶端失敗: {e}")
        return
    try:
        # logging.info("開始發送 GPT API 請求")
        completion = client.chat.completions.create(
            temperature=0,
            model="gpt-4o-0806",
            messages=[
                {
                    "role": "system",
                    "content": prompt
                },
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": user_input},
                    ],
                }
            ]
        )
        response_content = completion.choices[0].message.content
        prompt_tokens = completion.usage.prompt_tokens  # 提示 (prompt) token 數
        completion_tokens = completion.usage.completion_tokens  # 回應 (completion) token 數

        # logging.info(f"API 回應成功，Input token數量: {prompt_tokens}, Output token數量: {completion_tokens})")
        # 計算成本（按 1M tokens 計價）
        prompt_cost = (prompt_tokens / 1_000_000) * 2.5  # 2.5 美金 / 1M tokens
        completion_cost = (completion_tokens / 1_000_000) * 10  # 10 美金 / 1M tokens
        total_cost = prompt_cost + completion_cost
        # 顯示 token 消耗資訊
        print(f"Input Token: {prompt_tokens}")
        print(f"Output Token: {completion_tokens}")
        print(f"總共花費:{total_cost}美金")
        print(f"總共花費:{total_cost*32.85}台幣")
        # print(response_content)
        return response_content
    except Exception as e:
        logging.error(f"API 請求失敗: {e}")
        return

In [22]:
prompt = """ 
請幫我分析user輸入的評論在 Google 評論上的內容，並針對以下重點提供摘要：
1. **餐點口味**（總體評價、特色、常見評論）(10個字以內) 
2. **價位**（是否合理、性價比、與類似餐廳比較）
3. **推薦關鍵字**（常見的好評/負評關鍵詞）(10個字以內)  
4. **推薦的菜色**（評論中最常被提及的推薦菜品）(10個字以內)   
請依照以下範例格式輸出
   - 餐廳名稱：  
   - 總體評價（1~5分）：  
   - 餐點口味： 
   - 價位：  
   - 推薦關鍵字： 
   - 推薦的菜色：
"""

user_input ="""
11PASTA食義_南京店
覺得套餐的麵包非常好吃！
烤的酥酥脆脆的，一人一個有種意猶未盡的感覺。
義大利麵有多加一份蔬菜+30元
多了很多菇類，吃起來滿足

披薩點了燻鮭，覺得有點太鹹，但很喜歡披薩的「邊邊」酥香好吃，有舒服的麥味👍

套餐的沙拉算豐盛
美式比單點的美式稍小杯
甜點提拉米蘇還ok

1300來用餐依然客滿，建議先訂位
整體來說是會再回訪的餐廳。 
"""

In [23]:
res = GPT_API_text(prompt=prompt,user_input=user_input)

Input Token: 405
Output Token: 106
總共花費:0.0020724999999999997美金
總共花費:0.06808162499999999台幣


In [24]:
print(res)

- 餐廳名稱：11PASTA食義_南京店  
- 總體評價（1~5分）：4分  
- 餐點口味：麵包酥脆，披薩偏鹹  
- 價位：合理，性價比高  
- 推薦關鍵字：酥脆、滿足、客滿  
- 推薦的菜色：套餐麵包、燻鮭披薩


In [25]:
with open("google_Comment.md","w") as f:
    f.write(res)